# Exercício 01!

#### Aluno: **Pablo Ernesto Vigneaux Wilton**

**Entrega individual de um notebook com esses passos indentificados até dia 19/outubro.**

Com o dataset do projeto, faça o seguinte:
    
1. Crie uma pipeline específica para as seguintes variáveis e realize os tratamentos necessários para transformar essa informação em número ordenado:
    1. `NAME_EDUCATION_TYPE`
2. Crie uma pipeline específica para as seguintes variáveis, realizando um One Hot Encoder (leiam a documentação e setem o parâmetro drop conforme fizer sentido para vocês):
    1. `FLAG_OWN_CAR`
    2. `FLAG_OWN_REALTY`
    2. `NAME_INCOME_TYPE`
    1. `NAME_FAMILY_STATUS`
    1. `OCCUPATION_TYPE`
3. Crie uma pipeline específica para todas as outras variáveis que não são do tipo object. Adicione na pipeline a imputação de missing com a mediana. Com base nos próximos passos, responda a seguinte pergunta:
    1. **É necessário realizar o standard scaler nessas variáveis?**
4. Crie uma outra pipeline, que unirá todas as pipelines anteriores. Além de unir os steps passados, essa pipeline deve aplicar uma seleção de features baseada em um modelo de regressão logística. Resposta a seguinte pergunta:
    1. **Qual penalidade deve ser usada? Por que?**
5. Após a etapa de seleção de features da pipeline do passo anterior, adicione um modelo de árvore de decisão com `max_depth = 5` e treine o modelo, avaliando as métricas.
6. Faça uma busca em grid nos hiperparâmetros `min_samples_leaf` e `max_depth`, retreine o modelo e avalie as métricas. Responda a seguinte pergunta:
    1. **Houve ganho em performance? Se sim, de quanto?**
7. Substitua o modelo de árvore de decisão por um modelo XGBoost, AdaBoost ou LightGBM, realizando também uma busca nos hiperparâmetros (lembre-se de alterar o grid de hiperparâmetros a ser testado). Avalie:

- *Como ficou a performance do modelo usando o novo algoritmo?*
- *A variável `pipe_cat__OCCUPATION_TYPE_nan` está bonificando ou penalizando o cliente? Ou seja, se o cliente tiver ocupação como missing, ele tem tendência a ser melhor ou pior pagador?*    

## Inicializações

### Importação de bibliotecas

In [48]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import (train_test_split,
                                     GridSearchCV)
from sklearn.preprocessing import (OrdinalEncoder, 
                                   OneHotEncoder, 
                                   StandardScaler)
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

### Declaração de Constantes

In [49]:
ROOT = Path('./datasets')
DATA_FILE_NAME = 'application_train.csv'

## Carga de Dados

In [50]:
df = pd.read_csv(ROOT / DATA_FILE_NAME)

## Observação Inicial

In [51]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,456162,0,Cash loans,F,N,N,0,112500.0,700830.0,22738.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,134978,0,Cash loans,F,N,N,0,90000.0,375322.5,14422.5,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,3.0
2,318952,0,Cash loans,M,Y,N,0,180000.0,544491.0,16047.0,...,0,0,0,0,0.0,0.0,0.0,1.0,1.0,3.0
3,361264,0,Cash loans,F,N,Y,0,270000.0,814041.0,28971.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
4,260639,0,Cash loans,F,N,Y,0,144000.0,675000.0,21906.0,...,0,0,0,0,0.0,0.0,0.0,10.0,0.0,0.0


## Preparação do Conjunto de Dados e Principais Atributos

> *Atributos importantes como o `Id` (chave) do dataframe e o `target` devem possuir destaque 
> especial por possuirem usos diferenciados em algumas etapas*

In [52]:
var_id = 'SK_ID_CURR'
var_resp = 'TARGET'

> *A separação inicial dos conjuntos de dados deve ser a primeira atividade para que o conjunto > de `treinamento` não influencie no de `teste`, em alguns casos pode ser criado um terceiro 
> conjunto denominado `validação`, um dos motivos disto é o de prevenir a ocorrência de 
> `Data Leakage`*
> 
> *Obs.: Deve-se ter o cuidado de criar dataframes independentes fazendo uso da função `copy`, > isto devido a que o Python funciona internamente com ponteiros onde mais de uma variável pode > fazer referência ao mesmo objeto em memória*

In [53]:
df_treino, df_teste = train_test_split(df, test_size = 0.25, random_state = 1)

x_treino = df_treino.drop([var_resp, var_id], axis = 1).copy()
y_treino = df_treino[var_resp].copy()

x_teste = df_teste.drop([var_resp, var_id], axis = 1).copy()
y_teste = df_teste[var_resp].copy()

## Item 1

Crie uma pipeline específica para as seguintes variáveis e realize os tratamentos necessários para transformar essa informação em número ordenado:
- `NAME_EDUCATION_TYPE`

> *O atributo `NAME_EDUCATION_TYPE` se refere ao grau de educação e por isto é um atributo 
> `categórico` e `ordinal` significando que existe uma ordem ou hierarquia, algo que pode 
> influenciar no peso. Para este tipo de caso existe um tratamento diferenciado*

In [54]:
var_edu = ['NAME_EDUCATION_TYPE']

lista_ordenada = [
    'Lower secondary',
    'Secondary / secondary special', 
    'Incomplete higher',
    'Higher education', 
    'Academic degree'
]

pipe_edu = Pipeline(steps = [
    ('missing', SimpleImputer(missing_values='', strategy='most_frequent')),
    ('trata_edu', OrdinalEncoder(categories = [lista_ordenada]))
])

## Item 2

Crie uma pipeline específica para as seguintes variáveis, realizando um One Hot Encoder (leiam a documentação e setem o parâmetro drop conforme fizer sentido para vocês):
- `FLAG_OWN_CAR`
- `FLAG_OWN_REALTY`
- `NAME_INCOME_TYPE`
- `NAME_FAMILY_STATUS`
- `OCCUPATION_TYPE`

> *O `One Hot Encoder` é uma das abordagens possíveis para um preprocessamentos de atributos 
> `categóricos` do tipo `nominais`, transforamndo a coluna em um conjunto de colunas para cada > valor possível do atributo, preenchendo os valores com 0 ou 1 indicando se o valor em si se 
> aplica ou não. Para o caso de atributos binários, onde apenas existem dois valores possíveis, > a abordagem recomendada é a deleção de uma das colunas já que é impossívela a existência dos > dois valores ao mesmo tempo, isto pode ser executado via parâmetro `drop`.*
> 
> *A criação de uma `pipeline` é a abordagem mais comum de aplicação de um workflow, uma 
> automatização da execução de cada uma das etapas exigidas, existem diversas ferramentas para > a sua implementação, mas no caso utilizamos a existente no Sciki-learn.*

In [55]:
var_cat = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE', 
           'NAME_FAMILY_STATUS', 'OCCUPATION_TYPE']

pipe_cat = Pipeline(steps = [
    ('trata_cat', OneHotEncoder(sparse = False, drop = 'if_binary', handle_unknown='ignore'))
])

## Item 3

Crie uma pipeline específica para todas as outras variáveis que não são do tipo object. Adicione na pipeline a imputação de missing com a mediana. Com base nos próximos passos, responda a seguinte pergunta:
- *É necessário realizar o standard scaler nessas variáveis?*

> *A `pipeline` possui outras etapas de tratamento dos dados, incluindo a de tratamento dos 
> dados numéricos. Este tratamento inclui o `preenchimento` de dados faltantes, para este caso > a utilizando a `mediana`; e a transformação dos dados colocando-os na `mesma escala`, algo que
> evita entre outras coisas, que algum atributo possa ter um peso maior que o desejado, algo comum quando as faixas de valores é muito diferente entre atributos.*

In [56]:
var_num = df.select_dtypes(exclude = ['object']).drop(['TARGET', 'SK_ID_CURR'], axis = 1).columns.tolist()

pipe_num = Pipeline(steps = [
    ('trata_num', SimpleImputer(missing_values = np.nan, strategy = 'median')),
    ('scaler', StandardScaler())
])

## Item 4

Crie uma outra pipeline, que unirá todas as pipelines anteriores. Além de unir os steps passados, essa pipeline deve aplicar uma seleção de features baseada em um modelo de regressão logística. Responda a seguinte pergunta:
- *Qual penalidade deve ser usada? Por que?*

> *Como foram criadas `pipelines` específicas para algumas etapas, é importante que elas também > possam ser incluidas no workflow para serem executadas de forma automatizadas, por isto é
> criada uma `pipeline` que engloba todas as demais que sejam relativas ao preprocessamento e 
> quais quer outras que sejam convenientes.*
>
> *A seleção dos atributos a serem utilizados no modelo é realizada utilizando 
> `regressão  logística` através da função `LogisticRegressionCV`, tendo como principais
>  parâmetros:
> * **Cs**: Representando o inverso da força de regularização, quanto menor o valor maior a força, em suma é a força da `penalidade`.
>  No caso pode ser uma lista de valores; 
> * **solver**: Identifica o algoritmo utilizado na otimização, neste caso foi utilizado o 
> `'liblinear'` que é indicado para datasets pequenos;
> * **penalty**: Penaliza a regressão logistica por ter muitos atributos. Usada para
> especificar a norma utilizada na penalização. Os `solvers` não 
> suportam todos os tipos de `penalties`, no caso temos os `solvers` 'newton-cg', 'sag' e 
> 'lbfgs' que suportam apenas l2 penalidades. 'elasticnet' é suportado apenas pelo solver 
> 'saga'. 
>
> Para nosso caso L1 limita o tamanho dos coeficientes. L1 pode produzir modelos esparsos (ou 
> seja, modelos com poucos coeficientes); Alguns coeficientes podem se tornar zero e eliminados.*

In [57]:
pipe_preproc = ColumnTransformer(transformers = [
    ('pipe_edu', pipe_edu, var_edu),
    ('pipe_cat', pipe_cat, var_cat),
    ('pipe_num', pipe_num, var_num),
])

modelo_selecao = LogisticRegressionCV(
    penalty = 'l1', 
    solver = 'liblinear', 
    Cs = np.arange(0.01, 0.1), 
    scoring = 'roc_auc',
    random_state = 1
)

pipe_features = Pipeline(steps = [
    ('pipe_features', 
      SelectFromModel(estimator = modelo_selecao, 
                      max_features=10, 
                      threshold=0.1))
])

pipe_final = Pipeline(steps = [
    ('preproc', pipe_preproc),
    ('pipe_features', pipe_features)
])

*Visão do workflow da pipeline de seleção de atributos*

In [58]:
pipe_final

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('trata_edu',
                                                                   OrdinalEncoder(categories=[['Lower '
                                                                                               'secondary',
                                                                                               'Secondary '
                                                                                               '/ '
                                                                                               'secondary '
                                                                                               'special',
                                                                                               'Incomplete '
                                                                                               'higher',
                                                                                               'Higher '
                                                                                               'education',
                                                                                               'Academic '
                                                                                               'degree']]))]),
                                                  ['NAME_EDUCATION_TYPE']),
                                                 ('pipe_...
                                                   'LIVE_REGION_NOT_WORK_REGION',
                                                   'REG_CITY_NOT_LIVE_CITY',
                                                   'REG_CITY_NOT_WORK_CITY',
                                                   'LIVE_CITY_NOT_WORK_CITY',
                                                   'EXT_SOURCE_1',
                                                   'EXT_SOURCE_2',
                                                   'EXT_SOURCE_3', ...])])),
                ('pipe_features',
                 Pipeline(steps=[('pipe_features',
                                  SelectFromModel(estimator=LogisticRegressionCV(Cs=array([0.01]),
                                                                                 penalty='l1',
                                                                                 random_state=1,
                                                                                 scoring='roc_auc',
                                                                                 solver='liblinear'),
                                                  max_features=10,
                                                  threshold=0.1))]))])

> *Realizamod a execução da `pipeline` (que apesar de ter o nome de pipeline final ela apenas 
> vai até a etapa de seleção de atributos) para obter os atributos de maior importância.*

In [59]:
%%time
pipe_final.fit_transform(df, df['TARGET'])

CPU times: user 1min 1s, sys: 7.65 s, total: 1min 9s
Wall time: 1min 1s


array([[ 2.        ,  0.        ,  0.        , ...,  0.96944484,
        -1.96931659,  0.63787578],
       [ 1.        ,  0.        ,  0.        , ..., -1.65108648,
         1.44821535,  0.63787578],
       [ 1.        ,  1.        ,  0.        , ...,  0.99832013,
         0.63291717,  0.63787578],
       ...,
       [ 1.        ,  0.        ,  0.        , ..., -1.19881162,
         0.11212232, -1.56770336],
       [ 1.        ,  1.        ,  0.        , ..., -0.65266097,
         1.70630073, -1.56770336],
       [ 1.        ,  0.        ,  0.        , ...,  0.86594309,
         0.11212232,  0.63787578]])

## Item 5


Após a etapa de seleção de features da pipeline do passo anterior, adicione um modelo de árvore de decisão com `max_depth = 5` e treine o modelo, avaliando as métricas.

> *Adicionamos mais uma camada na pipeline final, camada esta que consiste do modelo preditivo 
> em si, onde será utilizada uma árvore de decisão com profundidade (altura) igual a 5*

In [60]:
modelo_previsao = DecisionTreeClassifier(max_depth = 5, 
                                         random_state = 1)

pipe_final = Pipeline(steps = [
    ('pipe_preproc', pipe_preproc),
    ('pipe_selecao', pipe_features),
    ('previsao', modelo_previsao)
])

> *Diagrama da pipeline final*

In [61]:
pipe_final

Pipeline(steps=[('pipe_preproc',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('trata_edu',
                                                                   OrdinalEncoder(categories=[['Lower '
                                                                                               'secondary',
                                                                                               'Secondary '
                                                                                               '/ '
                                                                                               'secondary '
                                                                                               'special',
                                                                                               'Incomplete '
                                                                                               'higher',
                                                                                               'Higher '
                                                                                               'education',
                                                                                               'Academic '
                                                                                               'degree']]))]),
                                                  ['NAME_EDUCATION_TYPE']),
                                                 ('...
                                                   'LIVE_CITY_NOT_WORK_CITY',
                                                   'EXT_SOURCE_1',
                                                   'EXT_SOURCE_2',
                                                   'EXT_SOURCE_3', ...])])),
                ('pipe_selecao',
                 Pipeline(steps=[('pipe_features',
                                  SelectFromModel(estimator=LogisticRegressionCV(Cs=array([0.01]),
                                                                                 penalty='l1',
                                                                                 random_state=1,
                                                                                 scoring='roc_auc',
                                                                                 solver='liblinear'),
                                                  max_features=10,
                                                  threshold=0.1))])),
                ('previsao',
                 DecisionTreeClassifier(max_depth=5, random_state=1))])

> *Execução da pipeline com os dados de treinamento, treinando o modelo.*

In [62]:
pipe_final.fit(x_treino, y_treino)

Pipeline(steps=[('pipe_preproc',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('trata_edu',
                                                                   OrdinalEncoder(categories=[['Lower '
                                                                                               'secondary',
                                                                                               'Secondary '
                                                                                               '/ '
                                                                                               'secondary '
                                                                                               'special',
                                                                                               'Incomplete '
                                                                                               'higher',
                                                                                               'Higher '
                                                                                               'education',
                                                                                               'Academic '
                                                                                               'degree']]))]),
                                                  ['NAME_EDUCATION_TYPE']),
                                                 ('...
                                                   'LIVE_CITY_NOT_WORK_CITY',
                                                   'EXT_SOURCE_1',
                                                   'EXT_SOURCE_2',
                                                   'EXT_SOURCE_3', ...])])),
                ('pipe_selecao',
                 Pipeline(steps=[('pipe_features',
                                  SelectFromModel(estimator=LogisticRegressionCV(Cs=array([0.01]),
                                                                                 penalty='l1',
                                                                                 random_state=1,
                                                                                 scoring='roc_auc',
                                                                                 solver='liblinear'),
                                                  max_features=10,
                                                  threshold=0.1))])),
                ('previsao',
                 DecisionTreeClassifier(max_depth=5, random_state=1))])

> *Realizando a predição nos dados de treinamento e de teste com base no modelo treinado*

In [63]:
y_treino_pred = pipe_final.predict_proba(x_treino)[:, 1]
y_teste_pred = pipe_final.predict_proba(x_teste)[:, 1]

> *Obtenção do escore ROC AUC dos dados de treinamento*

In [64]:
roc_auc_score(y_treino, y_treino_pred)

0.7129099120187238

> *Obtenção do escore ROC AUC dos dados de teste*

In [65]:
roc_auc_score(y_teste, y_teste_pred)

0.7028451303201754

> *Podemos ver que no caso do escore dos dados de treinamento o valor não foi muito alto, não é um modelo ótimo e também não ocorreu overfiting.*
>
> *Para o caso dos dados de teste o resultado é semelhante, muito próximo, algo que indica que está equilibrado.*

## Item 6

Faça uma busca em grid nos hiperparâmetros `min_samples_leaf` e `max_depth`, retreine o modelo e avalie as métricas. Responda a seguinte pergunta:
- *Houve ganho em performance? Se sim, de quanto?* 

> *Criação de estrutura de parâmetros para a execução do `GridSearchCV` aplicado a àrvore de
> decisão.*

In [66]:
parametros = {
    'previsao__max_depth': [10, 30, 50],
    'previsao__min_samples_leaf': [2, 3, 6]
}

gscv = GridSearchCV(
    estimator = pipe_final,
    param_grid = parametros,
    scoring = 'roc_auc',
    refit = True,
    cv = 3
)

In [67]:
gscv.fit(x_treino, y_treino)

/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encode

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('pipe_preproc',
                                        ColumnTransformer(transformers=[('pipe_edu',
                                                                         Pipeline(steps=[('missing',
                                                                                          SimpleImputer(missing_values='',
                                                                                                        strategy='most_frequent')),
                                                                                         ('trata_edu',
                                                                                          OrdinalEncoder(categories=[['Lower '
                                                                                                                      'secondary',
                                                                                                                      'Secondary '
                                                                                                                      '/ '
                                                                                                                      'secondary '
                                                                                                                      'special',
                                                                                                                      'Incomplete '
                                                                                                                      'higher',
                                                                                                                      'Higher '
                                                                                                                      'education',
                                                                                                                      'Academic '
                                                                                                                      'degree']]))])...
                                        Pipeline(steps=[('pipe_features',
                                                         SelectFromModel(estimator=LogisticRegressionCV(Cs=array([0.01]),
                                                                                                        penalty='l1',
                                                                                                        random_state=1,
                                                                                                        scoring='roc_auc',
                                                                                                        solver='liblinear'),
                                                                         max_features=10,
                                                                         threshold=0.1))])),
                                       ('previsao',
                                        DecisionTreeClassifier(max_depth=5,
                                                               random_state=1))]),
             param_grid={'previsao__max_depth': [10, 30, 50],
                         'previsao__min_samples_leaf': [2, 3, 6]},
             scoring='roc_auc')

In [68]:
gscv.best_params_

{'previsao__max_depth': 10, 'previsao__min_samples_leaf': 2}

In [69]:
gscv.best_estimator_

Pipeline(steps=[('pipe_preproc',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('trata_edu',
                                                                   OrdinalEncoder(categories=[['Lower '
                                                                                               'secondary',
                                                                                               'Secondary '
                                                                                               '/ '
                                                                                               'secondary '
                                                                                               'special',
                                                                                               'Incomplete '
                                                                                               'higher',
                                                                                               'Higher '
                                                                                               'education',
                                                                                               'Academic '
                                                                                               'degree']]))]),
                                                  ['NAME_EDUCATION_TYPE']),
                                                 ('...
                                                   'EXT_SOURCE_1',
                                                   'EXT_SOURCE_2',
                                                   'EXT_SOURCE_3', ...])])),
                ('pipe_selecao',
                 Pipeline(steps=[('pipe_features',
                                  SelectFromModel(estimator=LogisticRegressionCV(Cs=array([0.01]),
                                                                                 penalty='l1',
                                                                                 random_state=1,
                                                                                 scoring='roc_auc',
                                                                                 solver='liblinear'),
                                                  max_features=10,
                                                  threshold=0.1))])),
                ('previsao',
                 DecisionTreeClassifier(max_depth=10, min_samples_leaf=2,
                                        random_state=1))])

In [70]:
y_treino_pred = gscv.predict_proba(x_treino)[:, 1]
y_teste_pred = gscv.predict_proba(x_teste)[:, 1]

In [71]:
roc_auc_score(y_treino, y_treino_pred)

0.7534062659583587

In [72]:
roc_auc_score(y_teste, y_teste_pred)

0.7042631269674082

> *Houve uma pequena melhora na predição tanto de treinamento quanto de teste na segunda casa decimal ou mais, talvez devido aos warnings... Não pude identificar o origem destes.*

## Item Extra

Com a pipeline treinada, responda a seguinte pergunta:
- *Quantas e quais features são as que de fato são usadas pelo modelo? Por que?* 

In [73]:
features_out_logistica = pipe_final[:-1].get_feature_names_out()

In [74]:
pipe_final

Pipeline(steps=[('pipe_preproc',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('trata_edu',
                                                                   OrdinalEncoder(categories=[['Lower '
                                                                                               'secondary',
                                                                                               'Secondary '
                                                                                               '/ '
                                                                                               'secondary '
                                                                                               'special',
                                                                                               'Incomplete '
                                                                                               'higher',
                                                                                               'Higher '
                                                                                               'education',
                                                                                               'Academic '
                                                                                               'degree']]))]),
                                                  ['NAME_EDUCATION_TYPE']),
                                                 ('...
                                                   'LIVE_CITY_NOT_WORK_CITY',
                                                   'EXT_SOURCE_1',
                                                   'EXT_SOURCE_2',
                                                   'EXT_SOURCE_3', ...])])),
                ('pipe_selecao',
                 Pipeline(steps=[('pipe_features',
                                  SelectFromModel(estimator=LogisticRegressionCV(Cs=array([0.01]),
                                                                                 penalty='l1',
                                                                                 random_state=1,
                                                                                 scoring='roc_auc',
                                                                                 solver='liblinear'),
                                                  max_features=10,
                                                  threshold=0.1))])),
                ('previsao',
                 DecisionTreeClassifier(max_depth=5, random_state=1))])

In [75]:
features_used = pd.Series({i: j for i, j in zip(features_out_logistica, pipe_final[-1].feature_importances_)})

In [76]:
features_used.sort_values(ascending = False).head(10)

pipe_num__EXT_SOURCE_3                      0.465636
pipe_num__EXT_SOURCE_2                      0.453897
pipe_num__EXT_SOURCE_1                      0.073080
pipe_cat__OCCUPATION_TYPE_Drivers           0.004369
pipe_num__AMT_GOODS_PRICE                   0.003018
pipe_edu__NAME_EDUCATION_TYPE               0.000000
pipe_cat__FLAG_OWN_CAR_Y                    0.000000
pipe_cat__NAME_INCOME_TYPE_State servant    0.000000
pipe_num__AMT_CREDIT                        0.000000
pipe_num__FLAG_DOCUMENT_3                   0.000000
dtype: float64

> *Os atributos realmente utilizados foram:*
> 
> * EXT_SOURCE_3                      (0.465636)
> * EXT_SOURCE_2                      (0.453897)
> * EXT_SOURCE_1                      (0.073080)
> * OCCUPATION_TYPE_Drivers           (0.004369)
> * AMT_GOODS_PRICE                   (0.003018)

## Item 7 (Exercicio 2)

 Substitua o modelo de árvore de decisão por um modelo XGBoost ou LightGBM, realizando também uma busca nos hiperparâmetros (lembre-se de alterar o grid de hiperparâmetros a ser testado). Avalie:

- *Como ficou a performance do modelo usando o novo algoritmo?*
- *A variável `pipe_cat__OCCUPATION_TYPE_nan` está bonificando ou penalizando o cliente? Ou seja, se o cliente tiver ocupação como missing, ele tem tendência a ser melhor ou pior pagador?*

### Modelo LightGBM

In [77]:
from lightgbm import LGBMClassifier

In [78]:
# Criando modelo LightGBM
modelo_lgbm = LGBMClassifier(n_estimators=3)

parametros = {
    'modelo__n_estimators': [100, 300, 500],
}


pipe_final_lgbm = Pipeline(steps = [
    ('preproc', pipe_preproc),
    ('pipe_features', pipe_features),
    ('modelo', modelo_lgbm)
])

gscv_lgbm = GridSearchCV(
    estimator = pipe_final_lgbm,
    param_grid = parametros,
    scoring = 'roc_auc',
    refit = True,
    cv = 3
)

In [79]:
gscv_lgbm.fit(x_treino, y_treino)

/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:188: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preproc',
                                        ColumnTransformer(transformers=[('pipe_edu',
                                                                         Pipeline(steps=[('missing',
                                                                                          SimpleImputer(missing_values='',
                                                                                                        strategy='most_frequent')),
                                                                                         ('trata_edu',
                                                                                          OrdinalEncoder(categories=[['Lower '
                                                                                                                      'secondary',
                                                                                                                      'Secondary '
                                                                                                                      '/ '
                                                                                                                      'secondary '
                                                                                                                      'special',
                                                                                                                      'Incomplete '
                                                                                                                      'higher',
                                                                                                                      'Higher '
                                                                                                                      'education',
                                                                                                                      'Academic '
                                                                                                                      'degree']]))]),
                                                                         ['NA...
                                                                          'EXT_SOURCE_3', ...])])),
                                       ('pipe_features',
                                        Pipeline(steps=[('pipe_features',
                                                         SelectFromModel(estimator=LogisticRegressionCV(Cs=array([0.01]),
                                                                                                        penalty='l1',
                                                                                                        random_state=1,
                                                                                                        scoring='roc_auc',
                                                                                                        solver='liblinear'),
                                                                         max_features=10,
                                                                         threshold=0.1))])),
                                       ('modelo',
                                        LGBMClassifier(n_estimators=3))]),
             param_grid={'modelo__n_estimators': [100, 300, 500]},
             scoring='roc_auc')

In [80]:
gscv_lgbm.best_params_

{'modelo__n_estimators': 100}

In [81]:
gscv_lgbm.best_estimator_

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('trata_edu',
                                                                   OrdinalEncoder(categories=[['Lower '
                                                                                               'secondary',
                                                                                               'Secondary '
                                                                                               '/ '
                                                                                               'secondary '
                                                                                               'special',
                                                                                               'Incomplete '
                                                                                               'higher',
                                                                                               'Higher '
                                                                                               'education',
                                                                                               'Academic '
                                                                                               'degree']]))]),
                                                  ['NAME_EDUCATION_TYPE']),
                                                 ('pipe_...
                                                   'REG_CITY_NOT_LIVE_CITY',
                                                   'REG_CITY_NOT_WORK_CITY',
                                                   'LIVE_CITY_NOT_WORK_CITY',
                                                   'EXT_SOURCE_1',
                                                   'EXT_SOURCE_2',
                                                   'EXT_SOURCE_3', ...])])),
                ('pipe_features',
                 Pipeline(steps=[('pipe_features',
                                  SelectFromModel(estimator=LogisticRegressionCV(Cs=array([0.01]),
                                                                                 penalty='l1',
                                                                                 random_state=1,
                                                                                 scoring='roc_auc',
                                                                                 solver='liblinear'),
                                                  max_features=10,
                                                  threshold=0.1))])),
                ('modelo', LGBMClassifier())])

In [82]:
y_treino_pred = gscv_lgbm.predict_proba(x_treino)[:, 1]
y_teste_pred = gscv_lgbm.predict_proba(x_teste)[:, 1]

In [83]:
roc_auc_score(y_treino, y_treino_pred)

0.7776261760152745

In [84]:
roc_auc_score(y_teste, y_teste_pred)

0.7348742183972815

> *Uma pequena melhoria nos resultados:*
>
> |Dataset|DecisionTreeClassifier|LGBMClassifier    |
> |-------|----------------------|------------------|
> |treino |0.7534062659583587    |0.7776261760152745|
> |teste  |0.7042631269674082    |0.7348742183972815


### Modelo XGBoost

In [93]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [102]:
# Criando modelo xgboost

modelo_xgb = xgb.XGBClassifier(#objective ='multi:softprob', 
                               #colsample_bytree = 0.3, 
                               #learning_rate = 0.1,
                               #max_depth = 5, 
                               #alpha = 0.1, 
                               n_estimators = 10)


parametros = {
    'learning_rate': [0.1], #, 0.3]
}


gscv_xgb = GridSearchCV(
    estimator = modelo_xgb, # pipe_final_xgb
    param_grid = parametros,
    scoring = 'roc_auc',
    refit = True,
    cv = 3
)

pipe_final_xgb = Pipeline(steps = [
    ('preproc', pipe_preproc),
    ('pipe_features', pipe_features),
    ('grids', gscv_xgb) #('modelo', modelo_xgb)
])



In [103]:
pipe_final_xgb.fit(x_treino, y_treino)

/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:05:45] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:05:46] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:05:47] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/pabloernesto/anaconda3/envs/clusteringenv/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:05:50] WARNING: /tmp/abs_40obctay9q/croots/recipe/xgboost-split_1659548945886/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('pipe_edu',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(missing_values='',
                                                                                 strategy='most_frequent')),
                                                                  ('trata_edu',
                                                                   OrdinalEncoder(categories=[['Lower '
                                                                                               'secondary',
                                                                                               'Secondary '
                                                                                               '/ '
                                                                                               'secondary '
                                                                                               'special',
                                                                                               'Incomplete '
                                                                                               'higher',
                                                                                               'Higher '
                                                                                               'education',
                                                                                               'Academic '
                                                                                               'degree']]))]),
                                                  ['NAME_EDUCATION_TYPE']),
                                                 ('pipe_...
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      n_estimators=10,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=None,
                                                      reg_alpha=None,
                                                      reg_lambda=None,
                                                      scale_pos_weight=None,
                                                      subsample=None,
                                                      tree_method=None,
                                                      validate_parameters=None,
                                                      verbosity=None),
                              param_grid={'learning_rate': [0.1]},
                              scoring='roc_auc'))])

In [105]:
pipe_final_xgb[-1].best_params_

{'learning_rate': 0.1}

In [106]:
pipe_final_xgb[-1].best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=10, n_jobs=16,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [107]:
y_treino_pred = pipe_final_xgb.predict_proba(x_treino)[:, 1]
y_teste_pred = pipe_final_xgb.predict_proba(x_teste)[:, 1]

In [108]:
roc_auc_score(y_treino, y_treino_pred)

0.7339115719541369

In [109]:
roc_auc_score(y_teste, y_teste_pred)

0.7190591346789313

In [110]:
rmse = np.sqrt(mean_squared_error(y_teste, y_teste_pred))
print("RMSE: %f" % (rmse))

RMSE: 0.305293


> *Uma pequena melhoria nos resultados:*
>
> |Dataset|DecisionTreeClassifier|LGBMClassifier    |XGBRegressor|
> |-------|----------------------|------------------|------------|
> |treino |0.7534062659583587    |0.7776261760152745|
> |teste  |0.7042631269674082    |0.7348742183972815|
